Erdos-Renyi

In [6]:
import random
import io
import networkx as nx
import scipy as sp

n = 10000
p = 0.05
G = nx.fast_gnp_random_graph(n, p, directed=True)
print(G.number_of_edges())

for (u,v,w) in G.edges(data=True):
    w['weight'] = random.uniform(0, 0.6)

fh = open(f"./datasets/synthetic/er_{n}_{str(p).replace('.', '')}.mtx", "wb")
m = nx.to_scipy_sparse_array(G)
sp.io.mmwrite(fh, m, precision=5)

4999918


Scale Free

In [9]:
n = 10000
a = 0.41
b = 0.54
c = 0.05
G = nx.scale_free_graph(n, a, b, c)
print(G.number_of_edges())

for (u,v,w) in G.edges(data=True):
    w['weight'] = random.uniform(0, 0.6)

fh = open(f"./datasets/synthetic/sf_{n}_{str(a).replace('.', '')}_{str(b).replace('.', '')}_{str(c).replace('.', '')}.mtx", "wb")
m = nx.to_scipy_sparse_array(G)
sp.io.mmwrite(fh, m, precision=5)

21624


Higgs

In [10]:
fh = open("./graphs/higgs-social_network.edgelist", "rb")
G = nx.read_edgelist(fh, create_using=nx.DiGraph)
fh.close()

for (u,v,w) in G.edges(data=True):
    w['weight'] = random.uniform(0, 0.6)

fh = open(f"./datasets/real/higgs.mtx", "wb")
m = nx.to_scipy_sparse_array(G)
sp.io.mmwrite(fh, m, precision=5)

In [11]:
fh = open("./graphs/twitter_combined.txt", "rb")
G = nx.read_edgelist(fh, create_using=nx.DiGraph, nodetype=int)

for (u,v,w) in G.edges(data=True):
    w['weight'] = random.uniform(0, 0.6)

fh = open(f"./datasets/real/ego-twitter.mtx", "wb")
m = nx.to_scipy_sparse_array(G)
sp.io.mmwrite(fh, m, precision=5)

In [12]:
fh = open("./graphs/facebook_combined.txt", "rb")
G = nx.read_edgelist(fh, create_using=nx.Graph, nodetype=int)

G = G.to_directed()
for (u,v,w) in G.edges(data=True):
    w['weight'] = random.uniform(0, 0.6)

fh = open(f"./datasets/real/ego-facebook.mtx", "wb")
m = nx.to_scipy_sparse_array(G)
sp.io.mmwrite(fh, m, precision=5)